In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%load_ext cython

In [ ]:
import ROOT
import h5py
import os
from matplotlib import pyplot as plt
import numpy as np
import time
import sys
import datetime
import time

ROOT.gSystem.Load("$MARSSYS/libmars.so")

In [ ]:
filepath_in = "/remote/ceph/user/m/mhuetten/MAGIC_data/merpp/"
filepath_out = "/remote/ceph/user/m/mhuetten/MAGIC_data/ctapipe/"

filename_M1 = "20180821_M1_05074767.014_D_CrabNebula-W0.40+035.root"
filename_M2 = "20180821_M2_05074767.014_D_CrabNebula-W0.40+035.root"

filenames = [filename_M1, filename_M2]

infile_M1 = ROOT.TFile(os.path.join(filepath_in, filename_M1))
infile_M2 = ROOT.TFile(os.path.join(filepath_in, filename_M2))

infiles = [infile_M1, infile_M2]

filename_out = ''.join(filename_M1.split("_M1"))


In [ ]:
ROOT.TBrowser()

In [ ]:
%%cython

#from numba import jit, autojit
#@autojit

def ugly_loop(iter):
   
    cdef int i_pixel, nsample, pixid, fadc_val

    arr_2D = []
    arr_pixid = []
    
    for i_pixel in range(1184):
        arr_pixel = []
        iter.Jump(i_pixel)

        if iter.Jump(i_pixel):
            pixid = iter.GetPixelId()
            arr_pixid.append(pixid)            
            for nsample in range(iter.GetNumHiGainSamples()):
                fadc_val = iter.GetHiGainSampleVal(nsample)
                arr_pixel.append(fadc_val)
            arr_2D.append(arr_pixel)
    return arr_2D, arr_pixid

In [ ]:
f = h5py.File(os.path.join(filepath_out,filename_out.split(".root")[0] + ".hdf5"),"w")
dsets_events = []

for ntel in range(2):

    Header = ROOT.TChain("RunHeaders")
    Header.Add(os.path.join(filepath_in, filenames[ntel]))

    head = ROOT.MRawRunHeader()
    Header.SetBranchAddress("MRawRunHeader.",head)

    # extract global information out of first event:
    Header.GetEntry(0)
    TelescopeNumber =  head.GetTelescopeNumber()
    nevents = head.GetNumEvents()
    RunNumber = head.GetRunNumber()
    RunType = head.GetRunTypeStr()

    Events = ROOT.TChain("Events")
    Events.Add(os.path.join(filepath_in, filenames[ntel]))

    arr_3D = []
    arr_pixid = []
    arr_triggerid = []

    DAQEvtNumber = []
    ClockCounter = []
    NumTrigLvl2 = []
    StereoEvtNumber = []
    TriggerID = []
    TrigPattern = []
    CalibrationPattern = []
    MacrocellsPattern = []
    TimeDiff = []

    evtdata = ROOT.MRawEvtData()
    evtheader = ROOT.MRawEvtHeader()

    Events.SetBranchAddress("MRawEvtData.",evtdata)
    Events.SetBranchAddress("MRawEvtHeader.",evtheader)

    max_event = Events.GetEntries()
    #max_event = 2000

    start_time = time.time()

    for nevent in range(max_event):

        if nevent >= max_event:
           continue

        sys.stdout.write("\r ... processing event " + str(nevent) + ", " \
                         + "%s >>> Percent Done : %.2f%%" % (datetime.datetime.now(), 100*nevent/float(max_event)))


        Events.GetEvent(nevent)
        arr_triggerid.append(evtheader.GetTriggerID())

        DAQEvtNumber.append(evtheader.GetDAQEvtNumber())
        ClockCounter.append(evtheader.GetClockCounter())
        NumTrigLvl2.append(evtheader.GetNumTrigLvl2())

        StereoEvtNumber.append(evtheader.GetStereoEvtNumber())

        trigid = evtheader.GetTriggerID()
        TriggerID.append(trigid)
        
        binarypattern = '\t{0:016b}'.format(trigid&0xff) 
        
        triggpattern = (bool(int(binarypattern[-8])), 
                        bool(int(binarypattern[-1])), 
                        bool(int(binarypattern[-2])), 
                        bool(int(binarypattern[-3])), 
                        bool(int(binarypattern[-4])), 
                        bool(int(binarypattern[-5])), 
                        bool(int(binarypattern[-6])), 
                        bool(int(binarypattern[-7])))
         
        TrigPatternNames = []
        
        # valid for runs after 2013-12-13, see MARS MTriggerPatternDecode.cc, ll. 224        
        TrigPatternNames.append('L3 trigger')
        TrigPatternNames.append('Calibration')
        TrigPatternNames.append('LIDAR')
        TrigPatternNames.append('Pedestal')
        TrigPatternNames.append('Pulse injection')
        TrigPatternNames.append('Sumtrigger')
        TrigPatternNames.append('Topo trigger')
        TrigPatternNames.append('L1 trigger')
        TrigPattern.append(triggpattern)

        # check mono-character of run:
        #if nevent > 0:
         #   if triggpattern[0] == False:
         #       if StereoEvtNumber[-1] == StereoEvtNumber[-2]:
         #          StereoEvtNumber[-1] = -1
                #else:
                   # print "calib!"

        CalibrationPattern.append(evtheader.GetCalibrationPattern())
        MacrocellsPattern.append(evtheader.GetMacrocellsPattern())
        TimeDiff.append(evtheader.GetTimeDiff())

        npixels = evtdata.GetNumPixels()

        saturationval = evtdata.GetSaturationVal()
        name = evtdata.GetUniqueName()
        size = evtdata.Sizeof()

        iter = ROOT.MRawEvtPixelIter(evtdata)

        arr_2D, arr_pixid = ugly_loop(iter)

        arr_2D = np.array(arr_2D)
        arr_3D.append(arr_2D)

    end_time = time.time()
    elapsed = end_time - start_time
    print "\n File read into python arrays >>> Elapsed time: %.1f s" % elapsed
    
    arr_triggerid = np.array(arr_triggerid, dtype="H") 
    arr_pixid = np.array(arr_pixid, dtype="H")
    arr_3D = np.array(arr_3D, dtype="H")
    
    dset_name = "MAGIC" + str(ntel+1) + "_rawdata"
    

    dsets_events.append(f.create_dataset(os.path.join(dset_name,"Events"), data=arr_3D.transpose(), compression='gzip'))#, compression_opts=9)

    dsets_runheader = []
    dsets_runheader.append(f.create_dataset(os.path.join(dset_name,"RunHeader/PixelIDs"), data=arr_pixid))

    f.attrs['RunType'] = RunType
    f.attrs['NEvents'] = nevents
    f.attrs['RunNumber'] = RunNumber
   
    f[dset_name].attrs['TelescopeNumber'] = TelescopeNumber

    dsets_evtheader = []
                           
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/DAQEvtNumber"), data=DAQEvtNumber, dtype='H'))
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/ClockCounter"), data=ClockCounter))
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/NumTrigLvl2"), data=NumTrigLvl2))
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/StereoEvtNumber"), data=StereoEvtNumber))
    
    #comp_type = np.dtype([('Orbit', 'i'), ('Location', np.str_, 6), ('Temperature (F)', 'f8'), ('Pressure (inHg)', 'f8')])
    
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/TriggerID"), (max_event,), np.dtype([('Trigger ID', 'H')])))
    dsets_evtheader[-1][...] = TriggerID
    
    dt = np.dtype([(TrigPatternNames[0], '?'), (TrigPatternNames[1], '?'), (TrigPatternNames[2], '?'), 
                   (TrigPatternNames[3], '?'), (TrigPatternNames[4], '?'), (TrigPatternNames[5], '?'), 
                   (TrigPatternNames[6], '?'), (TrigPatternNames[7], '?')])
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/TrigPattern"), (max_event,), dt))
    dsets_evtheader[-1][...] = np.array(TrigPattern, dtype = dt)

    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/CalibrationPattern"), data=CalibrationPattern))
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/MacrocellsPattern"), data=MacrocellsPattern))
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/TimeDiff"), data=TimeDiff))

    dsets_events[ntel].attrs['Dimensions'] = ['Number of pixels (increasing pixel ID)', 'Number of samples', 'Number of events']

f.close()

In [ ]:
f.close()